## Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

Задание

1. Загрузите датасет. Описание датасета можно посмотреть здесь.
2. Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.




In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [2]:
data = pd.read_csv('SMSSpamCollection.txt', header=None, sep='\t', names = ['label','text'])

In [3]:
data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
#with open('SMSSpamCollection.txt', 'r') as f:
#    sms = f.read().splitlines()

3. Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".


In [11]:
sms_label_bi, sms_text = list([0 if i=='ham' else 1 for i in data["label"]]), list(data["text"])

In [42]:
#sms_label, sms_text = zip(*[line.split('\t') for line in sms])

In [43]:
#sms_label_bi = np.array([0 if t=='ham' else 1 for t in sms_label])

4. Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.


In [16]:
X_counts = CountVectorizer().fit_transform(sms_text)


5. Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [8]:
pipeline = Pipeline([('count_vect', CountVectorizer()), ('log_reg', LogisticRegression(random_state=2))])

In [12]:
random_state=2
scores = cross_val_score(pipeline, sms_text, sms_label_bi, scoring='f1', cv=10)

In [13]:
scores.mean()

0.9311542822856882

In [14]:
with open('ans1.txt', 'w') as f:
    ans1 = str(round(scores.mean(), 1))
    f.write(ans1)

6. А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.



In [15]:
sms_train = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
            "FreeMsg: Txt: claim your reward of 3 hours talk time",
            "Have you visited the last lecture on physics?",
            "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
            "Only 99$"]

In [17]:
pipeline.fit(sms_text, sms_label_bi)

Pipeline(steps=[('count_vect', CountVectorizer()),
                ('log_reg', LogisticRegression(random_state=2))])

In [18]:
sms_test_pr = pipeline.predict(sms_train)
sms_test_pr

array([1, 1, 0, 0, 0])

In [19]:
with open('ans2.txt', 'w') as f:
    ans2 = ' '.join(map(str, sms_test_pr))
    f.write(ans2)

7. Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.



In [20]:
ngram_range = [(2,2), (3,3), (1,3)]

scores = []
for ngram in ngram_range:
    pipeline = Pipeline([('count_vect', CountVectorizer(ngram_range=ngram)), ('log_reg', LogisticRegression())])
    score = cross_val_score(pipeline, sms_text, sms_label_bi, scoring='f1', cv=10).mean()
    scores.append(score)
    print('Ngram_range: ', ngram)
    print('Cross-val-score f1: %.4f\n' % score)


Ngram_range:  (2, 2)
Cross-val-score f1: 0.8169

Ngram_range:  (3, 3)
Cross-val-score f1: 0.7250

Ngram_range:  (1, 3)
Cross-val-score f1: 0.9217



In [21]:
with open('ans3.txt', 'w') as f:
    ans3 = ' '.join(map(str, ['%.2f' % sc for sc in scores]))
    f.write(ans3)

8. Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

 По какой-то причине  обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно  посчитайте частоты слов и обучите классификатор.  



In [22]:
scores = []
for ngram in ngram_range:
    pipeline = Pipeline([('count_vect', CountVectorizer(ngram_range=ngram)), ('mn_nb', MultinomialNB())])
    score = cross_val_score(pipeline, sms_text, sms_label_bi, scoring='f1', cv=10).mean()
    scores.append(score)
    print('Ngram_range: ', ngram)
    print('Cross-val-score f1: %.4f\n' % score)

Ngram_range:  (2, 2)
Cross-val-score f1: 0.9338

Ngram_range:  (3, 3)
Cross-val-score f1: 0.8711

Ngram_range:  (1, 3)
Cross-val-score f1: 0.9472



In [23]:
scores = []
for ngram in ngram_range:
    X_counts = CountVectorizer(ngram_range=ngram).fit_transform(sms_text)
    mn_nb = MultinomialNB()
    score = cross_val_score(mn_nb, X_counts, sms_label_bi, scoring='f1', cv=10).mean()
    scores.append(score)
    print('Ngram_range: ', ngram)
    print('Cross-val-score f1: %.4f\n' % score)

Ngram_range:  (2, 2)
Cross-val-score f1: 0.6458

Ngram_range:  (3, 3)
Cross-val-score f1: 0.3786

Ngram_range:  (1, 3)
Cross-val-score f1: 0.8885



In [24]:
with open('ans4.txt', 'w') as f:
    ans4 = ' '.join(map(str, ['%.2f' % sc for sc in scores]))
    f.write(ans4)

9. Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [25]:
pipeline = Pipeline([('tf_idf', TfidfVectorizer()),
                     ('log_reg', LogisticRegression())])
score = cross_val_score(pipeline, sms_text, sms_label_bi, scoring='f1', cv=10).mean()
print('Cross-val-score f1: %.4f\n' % score)

Cross-val-score f1: 0.8777



In [26]:
with open('ans5.txt', 'w') as f:
    ans5 = str(-1)
    f.write(ans5)